In [1]:
import os
import sys
import glob
import pickle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

In [2]:
#sys.path.append('/c/Users/mdous/Repo/personal/graph-label-propagation')
sys.path.append('/home/max/Repo/mdouskos/graph-label-propagation')
import utils.save_annotation as voc_save

In [3]:
flags = {'vis': False}

In [4]:
BASE_PATH = '/media/max/My Passport/Data/lp_data'
DATA_PATH = os.path.join(BASE_PATH, 'lp_full_500_20_1.0_0.0_1.0_1.0_True_0.95_0_False_0.01_500_0.5_512_dummy_False_0')
SLIC_PATH = os.path.join(DATA_PATH, 'fig/slic')
SCRIBBLE_PATH = os.path.join(BASE_PATH, 'scribble')
CONFIDENCE_PATH = os.path.join(DATA_PATH, 'Confidences')
LABEL_PATH = os.path.join(DATA_PATH, 'fig/pseudomask')
OUTPUT_PATH = os.path.join(DATA_PATH,'fig/pm_filt')
if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)
get_filename = lambda x: os.path.splitext(os.path.basename(x))[0]

In [5]:
def scribble_mask(imfile, dist=10, maskSize=5):
    sc_path = os.path.join(SCRIBBLE_PATH, os.path.basename(imfile))
    im = Image.open(sc_path)
    np_im = np.array(im, dtype=np.uint8)
    # scribble_pos = np.asarray(np_im < 255).nonzero()
    np_scribble = np.where(np_im < 255, 0, 255).astype(np.uint8)
    if dist>0:
        chamf = cv2.distanceTransform(np_scribble, cv2.DIST_C, maskSize)
        mask_d = np.where(chamf > dist, False, True)
    else:
        mask_d = np.where(np_im < 255,True, False)
    if flags['vis']:
        plt.imshow(mask_d)
    return mask_d

In [6]:
def valid_superpixels(imfile, mask_d):
    # find superpixels containing the scribbles
    slpath = os.path.join(SLIC_PATH, get_filename(imfile) + '_slic.npz.npy')
    slic_im = np.load(slpath)    
    superpixs = np.unique(slic_im[mask_d])  
    mask_sp = np.full(slic_im.shape, False, dtype=bool)
    for sp in superpixs:
        mask_sp[slic_im == sp] = True
    if flags['vis']:
        plt.imshow(slic_im)
        plt.imshow(mask_sp)
    return mask_sp

In [7]:
def confidence_mask(imfile):
    conffile = os.path.join(CONFIDENCE_PATH, os.path.basename(imfile))
    confidence = Image.open(conffile)
    np_conf = np.array(confidence, dtype=np.float)/65535.
    mask_conf = np.where(np_conf > 0.9, True, False)
    if flags['vis']:
        plt.imshow(mask_conf)
    return mask_conf

In [8]:
def filter_label(imfile, mask):
    labelfile = os.path.join(LABEL_PATH, get_filename(imfile) + '_pseudomask.png')
    label = Image.open(labelfile)
    np_label = np.array(label, dtype=np.uint8)
    filtered_label = np.where(mask,np_label,255)
    if flags['vis']:
        plt.imshow(filtered_label)
    return filtered_label

In [ ]:
combine_masks = lambda x: confidence_mask(x) & valid_superpixels(x, scribble_mask(x))

filelist = glob.glob(CONFIDENCE_PATH + "/*.png")
imfile = filelist[0]
mask = combine_masks(imfile)
filtered_label = filter_label(imfile, mask)

plt.subplot(2,1,1)
plt.imshow(mask)
plt.subplot(2,1,2)
plt.imshow(filtered_label)
# ref = Image.open(os.path.join(SCRIBBLE_PATH, os.path.basename(imfile)))
# print(np.any(np.abs(ref-filtered_label)>0))
# plt.imshow(np.abs(ref-filtered_label))
# voc_save.save_annotation_indexed(filtered_label, OUTPUT_PATH, get_filename(imfile))


In [9]:
for imfile in tqdm(filelist):
    # mask = confidence_mask(imfile) & valid_superpixels(imfile,scribble_mask(imfile, 0))
    # mask = valid_superpixels(imfile, scribble_mask(imfile, 0))
    mask = combine_masks(imfile)
    if flags['vis']:
        plt.imshow(mask)

    filtered_label = filter_label(imfile, mask)
    voc_save.save_annotation_indexed(filtered_label, OUTPUT_PATH, get_filename(imfile))

100%|██████████| 10582/10582 [20:29<00:00,  8.61it/s]
